# Predicting Housing Prices using Ames House Prices Dataset

In [2034]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2035]:
# Read data
df = pd.read_csv('AmesHousing.csv')

# Shuffle data with seed
df = df.sample(frac=1, random_state=0)

print(df.head(10))

      Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
2216   2217  909279080           50        RL           NaN     11275   Pave   
836     837  907126050           20        RL          65.0      9757   Pave   
2396   2397  528144030           60        RL          86.0     11065   Pave   
1962   1963  535452060           20        RL          70.0      7000   Pave   
305     306  911202100           50   C (all)          66.0      8712   Pave   
1139   1140  531382090           60        RL          65.0      8453   Pave   
727     728  902477130           30   C (all)          72.0      9392   Pave   
1663   1664  527402150           20        RL           NaN     10530   Pave   
2817   2818  908102320           20        RL         134.0     17755   Pave   
1511   1512  908276140           80        RL          62.0      7692   Pave   

     Alley Lot Shape Land Contour Utilities Lot Config Land Slope  \
2216   NaN       IR1          HLS    AllPub     Co

In [2036]:
def train_and_test(df, k=0):
    train_len = int(len(df) * 0.8)
    # Split data into train and test
    train = df[:train_len]
    test = df[train_len:]

    #Filter out features that are not numeric
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])

    # Filter out features
    features = numeric_train.columns.drop('SalePrice')

    # Instantiate model
    lr = LinearRegression()

    # Fit model
    lr.fit(train[features], train['SalePrice'])

    # Predict
    predictions = lr.predict(test[features])

    # Calculate MSE, RMSE and MAPE
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(test['SalePrice'], predictions)
    return mse, rmse, mape

mse, rmse, mape = train_and_test(df[['Gr Liv Area', 'SalePrice']])
print(f'MSE: {mse}, RMSE: {rmse}, MAPE: {mape}')

MSE: 3226672587.5796666, RMSE: 56803.80786161846, MAPE: 0.24888185016152842


# Dataset fixup

In [2037]:
print(f"Number of columns: {len(df.columns)}\n")
missing_cols = df.isnull().sum()
print(f"Columns with null values:\n{missing_cols.sort_values(ascending=False)}")

Number of columns: 82

Columns with null values:
Pool QC            2917
Misc Feature       2824
Alley              2732
Fence              2358
Fireplace Qu       1422
Lot Frontage        490
Garage Cond         159
Garage Finish       159
Garage Yr Blt       159
Garage Qual         159
Garage Type         157
Bsmt Exposure        83
BsmtFin Type 2       81
Bsmt Qual            80
Bsmt Cond            80
BsmtFin Type 1       80
Mas Vnr Area         23
Mas Vnr Type         23
Bsmt Full Bath        2
Bsmt Half Bath        2
BsmtFin SF 1          1
Garage Cars           1
Electrical            1
Total Bsmt SF         1
Bsmt Unf SF           1
BsmtFin SF 2          1
Garage Area           1
Paved Drive           0
Full Bath             0
Half Bath             0
Bedroom AbvGr         0
Kitchen AbvGr         0
Kitchen Qual          0
TotRms AbvGrd         0
Sale Condition        0
Sale Type             0
Yr Sold               0
Mo Sold               0
Misc Val              0
Functional     

In [2038]:
# Fill null Pool QC values with "-" if rows Pool Area is 0
df.loc[df['Pool Area'] == 0, 'Pool QC'] = '-'

# Fill null Misc Feature values with "-"
df['Misc Feature'] = df['Misc Feature'].fillna('-')
# Replace Misc Feature value "Othr" with "-"
df.loc[df['Misc Feature'] == 'Othr', 'Misc Feature'] = '-'

# Fill null Alley values with "-"
df['Alley'] = df['Alley'].fillna('-')

# Fill null Fence values with "-"
df['Fence'] = df['Fence'].fillna('-')

# Fill Fireplace Qu null values with "-" if rows Fireplaces is 0
df.loc[df['Fireplaces'] == 0, 'Fireplace Qu'] = '-'

# Fill null Lot Frontage values with mean
df['Lot Frontage'] = df['Lot Frontage'].fillna(df['Lot Frontage'].mean())

# Fill Garage values with "-" if rows Garage Area is 0
df.loc[df['Garage Area'] == 0, 'Garage Type'] = '-'
df.loc[df['Garage Area'] == 0, 'Garage Finish'] = '-'
df.loc[df['Garage Area'] == 0, 'Garage Qual'] = '-'
df.loc[df['Garage Area'] == 0, 'Garage Cond'] = '-'
df.loc[df['Garage Area'] == 0, 'Garage Yr Blt'] = df['Year Built']

In [2039]:
print(f"Number of columns: {len(df.columns)}\n")
missing_cols = df.isnull().sum()
print(f"Columns with null values:\n{missing_cols.sort_values(ascending=False)}")

Number of columns: 82

Columns with null values:
Bsmt Exposure      83
BsmtFin Type 2     81
BsmtFin Type 1     80
Bsmt Qual          80
Bsmt Cond          80
Mas Vnr Area       23
Mas Vnr Type       23
Garage Cond         2
Bsmt Half Bath      2
Garage Yr Blt       2
Garage Finish       2
Garage Qual         2
Bsmt Full Bath      2
Electrical          1
BsmtFin SF 1        1
Total Bsmt SF       1
Garage Area         1
Garage Cars         1
BsmtFin SF 2        1
Bsmt Unf SF         1
Half Bath           0
Full Bath           0
Bedroom AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Kitchen AbvGr       0
Order               0
Fireplaces          0
Pool QC             0
Sale Condition      0
Sale Type           0
Yr Sold             0
Mo Sold             0
Misc Val            0
Misc Feature        0
Fence               0
Pool Area           0
Fireplace Qu        0
Screen Porch        0
3Ssn Porch          0
Enclosed Porch      0
Open Porch SF       0
Wood 

In [2040]:
print(f"Number of rows: {len(df)}\n")
# Drop rows with null values(there aren't many rows)
df = df.dropna(axis=0)
print(f"Number of rows after dropping null values: {len(df)}")

Number of rows: 2930

Number of rows after dropping null values: 2820


In [2041]:
# Add useful features
building_age = df['Yr Sold'] - df['Year Built']
remod_age = df['Yr Sold'] - df['Year Remod/Add']
garage_age = df['Yr Sold'] - df['Garage Yr Blt']

df['Building Age'] = building_age
df['Remod Age'] = remod_age
df['Garage Age'] = garage_age

# Drop invalid values
df = df[df['Building Age'] >= 0]
df = df[df['Remod Age'] >= 0]

# Drop columns that are not useful for prediction
df = df.drop(['PID', 'Order', 'Mo Sold', 'Garage Yr Blt', 'Yr Sold', 'Year Built', 'Year Remod/Add', 'Misc Val'], axis=1)

Test with only numeric values

In [2042]:
def train_and_test(df):
    # Split data into train and test
    train_len = int(len(df) * 0.8)
    train = df[:train_len]
    test = df[train_len:]

    #Filter out features that are not numeric
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])

    # Filter out features
    features = numeric_train.columns.drop('SalePrice')

    # Instantiate model
    lr = LinearRegression()

    # Fit model
    lr.fit(train[features], train['SalePrice'])

    # Predict
    predictions = lr.predict(test[features])

    # Calculate MSE, RMSE and MAPE
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(test['SalePrice'], predictions)
    return mse, rmse, mape

mse, rmse, mape = train_and_test(df)
print(f'MSE: {mse}, RMSE: {rmse}, MAPE: {mape}')

MSE: 795196444.6898594, RMSE: 28199.227732153577, MAPE: 0.1168394971595438


# Numerical Feature Selection

In [2043]:
nominal_features = ["MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
# Drop string columns that are not nominal features
df = df.drop(df.select_dtypes(include=['object']).columns.drop(nominal_features, errors="ignore"), axis=1)

#Convert nominal features to categorical columns
for col in nominal_features:
    df[col] = df[col].astype('category')

# Drop features with non unique values
nominal_features_non_uniq = [col for col in nominal_features if df[col].nunique() > 9]
df = df.drop(nominal_features_non_uniq, axis=1)
nominal_features = [col for col in nominal_features if col not in nominal_features_non_uniq]

print(f"Deleted nominal features: {nominal_features_non_uniq}")

corr_coeffs = df.corr(numeric_only=True)['SalePrice'].abs().sort_values(ascending=False)

# Delete non nominal and numeric features with correlation coefficient less than 0.1
low_coeff_features = corr_coeffs[corr_coeffs < 0.1].index
low_coeff_cols = df[low_coeff_features].columns
low_coeff_num_cols = df[low_coeff_cols].select_dtypes(include=['integer', 'float']).columns
low_coeff_non_nominal_cols = low_coeff_num_cols.drop(nominal_features, errors='ignore')
df = df.drop(low_coeff_non_nominal_cols, axis=1)
print(f"Deleted numeric features with correlation coefficient less than 0.45:\n{low_coeff_non_nominal_cols}")

# List features
features = df.columns.drop('SalePrice')
print(f"Features: {features}")
print (f"Number of features: {len(features)}")

# Test model
mse, rmse, mape = train_and_test(df.copy().drop(nominal_features, errors='ignore'))
print(f'MSE: {mse}, RMSE: {rmse}, MAPE: {mape}')



Deleted nominal features: ['MS SubClass', 'Neighborhood', 'Exterior 1st', 'Exterior 2nd', 'Sale Type']
Deleted numeric features with correlation coefficient less than 0.45:
Index(['Kitchen AbvGr', 'Pool Area', 'Bsmt Half Bath', 'Low Qual Fin SF',
       '3Ssn Porch', 'BsmtFin SF 2'],
      dtype='object')
Features: Index(['MS Zoning', 'Lot Frontage', 'Lot Area', 'Street', 'Alley',
       'Land Contour', 'Lot Config', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Mas Vnr Type', 'Mas Vnr Area', 'Foundation',
       'BsmtFin SF 1', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating',
       'Central Air', '1st Flr SF', '2nd Flr SF', 'Gr Liv Area',
       'Bsmt Full Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'TotRms AbvGrd', 'Fireplaces', 'Garage Type', 'Garage Cars',
       'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       'Screen Porch', 'Misc Feature', 'Sale Condition', 'Buil

# Categorical feature selection

In [2044]:
# Create dummy columns and add back to the dataframe
df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)

# Drop original categorical columns
df = df.drop(nominal_features, axis=1)

print(f"Features: {df.columns.drop('SalePrice')}")
print (f"Number of features: {len(df.columns.drop('SalePrice'))}")

# Test model
mse, rmse, mape = train_and_test(df)
print(f'MSE: {mse}, RMSE: {rmse}, MAPE: {mape}')

Features: Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Mas Vnr Area', 'BsmtFin SF 1', 'Bsmt Unf SF', 'Total Bsmt SF',
       '1st Flr SF', '2nd Flr SF',
       ...
       'Misc Feature_-', 'Misc Feature_Gar2', 'Misc Feature_Shed',
       'Misc Feature_TenC', 'Sale Condition_Abnorml', 'Sale Condition_AdjLand',
       'Sale Condition_Alloca', 'Sale Condition_Family',
       'Sale Condition_Normal', 'Sale Condition_Partial'],
      dtype='object', length=123)
Number of features: 123
MSE: 664494479.2377356, RMSE: 25777.790425824624, MAPE: 0.10926520732541865
